# Topological Searches in pandapower

This is an introduction into the honeypot module topology. The topology module provides topoligical searches and analyses for pandapower networks based on the NetworkX library. This tutorial will show you how to get started and demonstrate a few use cases of the module. For a full documentation of the topology functions, see the pandapower documentation.

To demonstrate the usage of the topology module we will use the medium voltage open ring from the simple pandapower test networks:

In [65]:
import pandapower.networks as nw
net = nw.simple_mv_open_ring_net()

## Creating MultiGraphs

The pandapower topology package provides a function to translate a pandapower network into an [networkx multigraph](https://networkx.github.io/documentation/networkx-1.9.1/reference/classes.multigraph.html):

In [66]:
import pandapower.topology as top
mg = top.create_nxgraph(net) # converts example network into a MultiGraph 

This picture visualises the conversion: On the left hand side you can see what our example network looks like, on the right hand side how it gets converted into a MultiGraph.

<img src="pics/multigraph_example.png">
    


## Algorithms from the NetworkX package

The bus numbers in the networkx graph are the same as the bus indices in pandapower. You can now use all [networkx algorithms](https://networkx.github.io/documentation/networkx-1.9.1/reference/algorithms.html) to perform graph searches on the network.

### shortest path

To find the [shortest path](https://networkx.github.io/documentation/networkx-1.9.1/reference/algorithms.shortest_paths.html) between nodes bus0 and bus5:

<img src="pics/nx_shortest_path.png">

In [73]:
import networkx as nx
path = nx.shortest_path(mg, 0, 5)
path

[0, 1, 6, 5]

This will algorithm will find the shortest path in terms of number of visited buses. The length of the lines is encoded in the weight parameter of the edges. If we want to find the shortest path in terms of shortest line length, we have to pass the weight parameter to the search:

In [77]:
path = nx.shortest_path(mg, 0, 5, weight="weight")
path

[0, 1, 6, 5]

In this case the search of course yields the same path, since there is only one path from bus0 to bus5.

Since the bus indices in the graph and in the pandapower network are the same, we can use the path to directly access buses in pandapower:

In [79]:
net.bus.loc[path]

,name,vn_kv,type,zone,in_service
0,110 kV bar,110.0,b,None,True
1,20 kV bar,20.0,b,None,True
6,bus 6,20.0,b,None,True
5,bus 5,20.0,b,None,True


gives us all buses on the shortest path between bus0 and bus5. We can also use the bus indices to find branch elements directly in pandapower. For example, to find all lines on the path:

In [80]:
net.line[(net.line.from_bus.isin(path)) & (net.line.to_bus.isin(path))]

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,imax_ka,df,parallel,type,in_service
4,line 4,NA2XS2Y 1x185 RM/25 12/20 kV,5,6,1.0,0.161,0.117,273.0,0.362,1.0,1,cs,True
5,line 5,NA2XS2Y 1x185 RM/25 12/20 kV,6,1,1.0,0.161,0.117,273.0,0.362,1.0,1,cs,True


or all transformers on this path:

In [81]:
net.trafo[(net.trafo.hv_bus.isin(path)) & (net.trafo.lv_bus.isin(path))]

,name,std_type,hv_bus,lv_bus,sn_kva,vn_hv_kv,vn_lv_kv,vsc_percent,vscr_percent,pfe_kw,i0_percent,shift_degree,tp_side,tp_mid,tp_min,tp_max,tp_st_percent,tp_pos,in_service
0,None,25 MVA 110/20 kV,0,1,25000.0,110.0,20.0,11.2,0.282,29.0,0.071,150.0,hv,0,-9,9,1.5,0,True


### customizing graph conversion

Now suppose we want to find the shortest path distance between bus2 and bus6 without going through the transformer substation, but allowing to go over open switches. The path we are looking for is therefore bus6 --> bus5 --> bus4 --> bus3 --> bus2.

This is not a path in the graph above though, since there is no edge between bus4 and bus5. We translate the graph with respect_switches=False to include the line with an open switch in the graph:

In [82]:
mg = top.create_nxgraph(net, respect_switches=False)

<img src="pics/multigraph_example_respect_switches.png">

Now we still have the problem that the shortest path algorithm will find the path over the substation (bus1) as the shortest path:

In [83]:
nx.shortest_path(mg, 6, 2)

[6, 1, 2]

To prevent this, we can specify bus1 as a nogobus in the conversion, which means it will not be translated to the networkx graph:

In [84]:
mg = top.create_nxgraph(net, respect_switches=False, nogobuses={1})

Now we get the path that we were looking for:

In [85]:
nx.shortest_path(mg, 6, 2)

[6, 5, 4, 3, 2]

### cycles

We can also use the [cycle algorithms](https://networkx.github.io/documentation/networkx-1.9.1/reference/algorithms.cycles.html) to find cycles in the network. Cycle algorithms only work on undirected graphs, which is why we need to specify multi=False in the graph conversion:

In [102]:
mg = top.create_nxgraph(net, multi=False)
nx.cycle_basis(mg)

[]

There are no cycles in the network, which confirms the radiality of the network. If we do not respect the switches, we will find the ring as a cycle:

In [103]:
mg = top.create_nxgraph(net, respect_switches=False, multi=False)
nx.cycle_basis(mg)

[[2, 3, 4, 5, 6, 1]]

## Algorithms in the topology package

Besides from using networkx algorithms, there are some custom algorithms in the pandapower.topolgoy package. For a full list with explanation see the pandapower documentation. Here, we only cover the two most important ones: connected_component and connected_components.

### Connected component

The connected component function returns all buses that are connected to a bus in the networkx graph. Suppose we want to find all buses that are on the same feeder as bus 2. We set bus1 as a nogobus and search for all buses connected to bus 2:

In [55]:
mg = top.create_nxgraph(net, nogobuses={1})
area = top.connected_component(mg, 2)

This generator contains all buses connected to bus2:

In [56]:
set(area)

{2, 3, 4}

We get the buses 2,3 and 4, but not bus1, since it was defined as a nogobus. If we want to get bus1 as connected to bus2, but still not go over bus2, we can define bus1 as a notravbus instead of a nogobus. This means that search algorithms will find the bus as connected, but not traverse it:

In [57]:
mg = top.create_nxgraph(net, notravbuses={1})
set(top.connected_component(mg, 2))

{1, 2, 3, 4}

### Connected components

If we don't want to find the area connected to one specific bus, but rather all areas that are connected, we can use the connected_components function:

In [58]:
mg = top.create_nxgraph(net, nogobuses={0, 1})
for area in top.connected_components(mg):
    print(area)

{2, 3, 4}
{5, 6}


Once again, we can alternatively use notravbuses to get the substation bus in the areas:

In [60]:
mg = top.create_nxgraph(net, nogobuses={0}, notravbuses={1})
for area in top.connected_components(mg):
    print(area)

{1}
{1, 2, 3, 4}
{1, 5, 6}


If we want to avoid getting the notravbus as an own area, we can alternatively pass the notravbuses argument directly to the connected_components search instead of to the graph conversion:

In [61]:
mg = top.create_nxgraph(net, nogobuses={0})
for area in top.connected_components(mg, notravbuses={1}):
    print(area)

{1, 2, 3, 4}
{1, 5, 6}


For more examples of 